In [26]:
import math
import numpy as np
from copy import copy
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch import nn

In [2]:
B = 64      # 배치 사이즈
M = 10      # 토큰의 최대 길이
V = 1024    # 토큰의 개수
N = 8       # 멀티헤드 개수
H = 512     # 토큰의 임베딩 사이즈
EXP = 2048  # 확장 사이즈 (FeedForward 클래스 참고)
L = 6       # 인코더/디코더 레이어 개수

In [3]:
def attention(query, key, value, scale):
    score = torch.matmul(query, key.transpose(-2, -1)) / scale
    prob = F.softmax(score, dim=-1)
    attn = torch.matmul(prob, value)
    return attn

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_head, hidden_size):
        super(MultiHeadAttention, self).__init__()
        self.num_head = num_head
        self.dk = hidden_size // self.num_head
    
    def forward(self, query, key, value):
        '''
        x = torch.rand((B, M, H))
        m = MultiHeadAttention(N, H)
        v = m(x, x, x)
        v.shape  # torch.Size([64, 10, 512])
        '''
        n_batch = query.shape[0]
        query = query.view(n_batch, -1, self.num_head, self.dk).transpose(1, 2)
        key = key.view(n_batch, -1, self.num_head, self.dk).transpose(1, 2)
        value = value.view(n_batch, -1, self.num_head, self.dk).transpose(1, 2)
        
        x = attention(query, key, value, self.dk)
        x = x.transpose(1, 2).contiguous().view(n_batch, -1, self.dk * self.num_head)
        return x

In [5]:
x = torch.rand((B, M, H))
m = MultiHeadAttention(N, H)
v = m(x, x, x)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [6]:
class FeedForward(nn.Module):
    def __init__(self, hidden_size, expand_size):
        super(FeedForward, self).__init__()
        self.linear_1 = nn.Linear(hidden_size, expand_size)
        self.linear_2 = nn.Linear(expand_size, hidden_size)
        
    def forward(self, x):
        '''
        x = torch.rand((B, M, H))
        m = FeedForward(H, EXP)
        v = m(x)
        v.shape  # torch.Size([64, 10, 512])
        '''
        x = self.linear_1(x)
        x = self.linear_2(x)
        return x

In [7]:
x = torch.rand((B, M, H))
m = FeedForward(H, EXP)
v = m(x)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [8]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_size):
        super(EncoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(N, hidden_size)
        self.feedforward = FeedForward(hidden_size, EXP)
        
    def forward(self, x):
        '''
        x = torch.rand((B, M, H))
        m = EncoderLayer(H)
        v = m(x)
        v.shape  # torch.Size([64, 10, 512])
        '''
        x = self.self_attention(x, x, x)
        x = self.feedforward(x)
        return x

In [9]:
x = torch.rand((B, M, H))
m = EncoderLayer(H)
v = m(x)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [10]:
class Embedding(nn.Module):
    def __init__(self, n_vocab, hidden_size):
        super(Embedding, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(n_vocab, hidden_size)
        
    def forward(self, x):
        '''
        data = np.random.randint(0, V, (B, M))
        x = torch.from_numpy(data)
        m = Embedding(V, H)
        v = m(x)
        v.shape  # torch.Size([64, 10, 512])
        '''
        return self.embedding(x)

In [11]:
data = np.random.randint(0, V, (B, M))
x = torch.from_numpy(data)
m = Embedding(V, H)
v = m(x)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [12]:
class Encoder(nn.Module):
    def __init__(self, n_layers):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.embedding = Embedding(V, H)
        self.layers = [EncoderLayer(H) for i in range(n_layers)]
    
    def forward(self, x):
        '''
        data = np.random.randint(0, V, (B, M))
        x = torch.from_numpy(data)
        m = Encoder(L)
        v = m(x)
        v.shape  # torch.Size([64, 10, 512])
        '''
        x = self.embedding(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [13]:
data = np.random.randint(0, V, (B, M))
x = torch.from_numpy(data)
m = Encoder(L)
v = m(x)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [14]:
class DecoderLayer(nn.Module):
    def __init__(self, n_head, hidden_size):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(n_head, hidden_size)
        self.encdec_attention = MultiHeadAttention(n_head, hidden_size)
        self.feedforward = FeedForward(hidden_size, 2048)
        
    def forward(self, x, memory):
        '''
        x = torch.rand((B, M, H))
        mem = copy(x)
        m = DecoderLayer(N, H)
        v = m(x, mem)
        v.shape  # torch.Size([64, 10, 512])
        '''
        x = self.self_attention(x, memory, memory)
        return x

In [15]:
x = torch.rand((B, M, H))
mem = copy(x)
m = DecoderLayer(N, H)
v = m(x, mem)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [16]:
class Decoder(nn.Module):
    def __init__(self, n_layers):
        super(Decoder, self).__init__()
        self.embedding = Embedding(V, H)
        self.layers = [DecoderLayer(N, H) for i in range(n_layers)]
        
    def forward(self, x, memory):
        '''
        data = np.random.randint(0, V, (B, M))
        x = torch.from_numpy(data)
        mem = torch.rand((B, M, H))
        m = Decoder(L)
        v = m(x, mem)
        v.shape  # torch.Size([64, 10, 512])
        '''
        x = self.embedding(x)
        for layer in self.layers:
            x = layer(x, memory)
        return x

In [17]:
data = np.random.randint(0, V, (B, M))
x = torch.from_numpy(data)
mem = torch.rand((B, M, H))
m = Decoder(L)
v = m(x, mem)
v.shape  # 

torch.Size([64, 10, 512])

In [18]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder(L)
        self.decoder = Decoder(L)
        
    def forward(self, src, dst):
        '''
        data = np.random.randint(0, V, (B, M))
        src = torch.from_numpy(data)
        data = np.random.randint(0, V, (B, M))
        dst = torch.from_numpy(data)
        src.shape, dst.shape

        m = Transformer()
        v = m(src, dst)
        v.shape  # torch.Size([64, 10, 512])
        '''
        src_encoded = self.encoder(src)
        dst_decoded = self.decoder(dst, src_encoded)
        
        return dst_decoded

In [19]:
data = np.random.randint(0, V, (B, M))
src = torch.from_numpy(data)
data = np.random.randint(0, V, (B, M))
dst = torch.from_numpy(data)
src.shape, dst.shape

m = Transformer()
v = m(src, dst)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [44]:
class PositionalEncoding(nn.Module):
    def __init__(self, hidden_size):
        super(PositionalEncoding, self).__init__()
        pos_encoding = torch.zeros(M, hidden_size)
        position = torch.arange(0, M).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_size, 2) *
                             -(math.log(10000.0) / hidden_size))
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        self.pos_encoding = pos_encoding.unsqueeze(0)
        
    def forward(self, x):
        '''
        x = torch.rand((B, M, H))
        m = PositionalEncoding(H)
        v = m(x)
        v.shape  # torch.Size([64, 10, 512])
        '''
        x = x + Variable(self.pos_encoding[:, :x.size(1)], 
                         requires_grad=False)
        return x

In [43]:
x = torch.rand((B, M, H))
m = PositionalEncoding(H)
v = m(x)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [45]:
class Encoder(nn.Module):
    def __init__(self, n_layers):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.embedding = Embedding(V, H)
        self.position = PositionalEncoding(H)
        self.layers = [EncoderLayer(H) for i in range(n_layers)]
    
    def forward(self, x):
        '''
        data = np.random.randint(0, V, (B, M))
        x = torch.from_numpy(data)
        m = Encoder(L)
        v = m(x)
        v.shape  # torch.Size([64, 10, 512])
        '''
        x = self.embedding(x)
        x = self.position(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [47]:
data = np.random.randint(0, V, (B, M))
x = torch.from_numpy(data)
m = Encoder(L)
v = m(x)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [48]:
class Decoder(nn.Module):
    def __init__(self, n_layers):
        super(Decoder, self).__init__()
        self.embedding = Embedding(V, H)
        self.position = PositionalEncoding(H)
        self.layers = [DecoderLayer(N, H) for i in range(n_layers)]
        
    def forward(self, x, memory):
        '''
        data = np.random.randint(0, V, (B, M))
        x = torch.from_numpy(data)
        mem = torch.rand((B, M, H))
        m = Decoder(L)
        v = m(x, mem)
        v.shape  # torch.Size([64, 10, 512])
        '''
        x = self.embedding(x)
        x = self.position(x)
        for layer in self.layers:
            x = layer(x, memory)
        return x

In [49]:
data = np.random.randint(0, V, (B, M))
x = torch.from_numpy(data)
mem = torch.rand((B, M, H))
m = Decoder(L)
v = m(x, mem)
v.shape  # torch.Size([64, 10, 512])

torch.Size([64, 10, 512])

In [50]:
### BLEU

In [73]:
from nltk import ngrams
from collections import Counter

In [74]:
def word_counter(tokens, n):
    return Counter(ngrams(tokens, n))

In [75]:
def modified_count(candidate, references, n):
    cand_cnt = word_counter(candidate.split(), n)
    temp = Counter()
    for ref in references:
        tokens = ref.split()
        ref_cnt = word_counter(tokens, n)
        for k, v in ref_cnt.items():
            if k in temp:
                temp[k] = max(ref_cnt[k], temp[k])
            else:
                temp[k] = ref_cnt[k]
    
    return Counter({tok:min(temp[tok], cand_cnt[tok]) for tok in cand_cnt})


In [76]:
def modified_ngram_precision(candidate, references, n):
    min_cnt = modified_count(candidate, references, n)
    total_cnt = word_counter(candidate.split(), n)
    
    min_cnt_sum = sum(min_cnt.values())
    total_cnt_sum = sum(total_cnt.values())
    #print('modified {}-gram precision: {}/{}'.format(n, min_cnt_sum, total_cnt_sum))
    return min_cnt_sum / total_cnt_sum

In [77]:
references = [
    'You know, I always love you so much',
    'Know right? I always love you a lot',
]

In [78]:
def brevity_penalty(candidate, references):
    candidate = candidate.lower()
    cand_len = len(candidate.split())
    diff = np.inf
    for ref in references:
        ref = ref.lower()
        tokens = ref.split()
        if diff > abs(len(tokens) - cand_len):
            ref_len = len(tokens)
            diff = abs(len(tokens) - cand_len)
            
    if cand_len > ref_len:
        return 1
    elif cand_len == 0:
        return 0
    else:
        return np.exp(1 - ref_len/cand_len)

In [102]:
def bleu_score(candidate, reference_list):
    candidate = candidate.lower()
    cand_len = len(candidate.split())
    weights = [1/cand_len] * cand_len
    bp = brevity_penalty(candidate, references)
    precisions = [modified_ngram_precision(candidate, references, n=i+1) for i in range(len(weights))] 
    
    score = 0
    for w, p in zip(weights, precisions):
        print(w, p, score)
        if p == 0:
            continue
        score += (w * np.log(p))
    
    print(bp)
    return bp * np.exp(score)

In [103]:
cand = 'know that I always love you that much.'
bleu_score(cand, references)

0.125 0.375 0
0.125 0.2857142857142857 -0.12260365662646577
0.125 0.16666666666666666 -0.2791990276883868
0.125 0.0 -0.5031689613418937
0.125 0.0 -0.5031689613418937
0.125 0.0 -0.5031689613418937
0.125 0.0 -0.5031689613418937
0.125 0.0 -0.5031689613418937
1.0


0.6046116297711386

In [104]:
cand = 'you love I always.'
bleu_score(cand, references)

0.25 0.5 0
0.25 0.0 -0.17328679513998632
0.25 0.0 -0.17328679513998632
0.25 0.0 -0.17328679513998632
0.36787944117144233


0.30934850332660563

In [105]:
cand = 'I I I I I.'
bleu_score(cand, references)

0.2 0.0 0
0.2 0.0 0
0.2 0.0 0
0.2 0.0 0
0.2 0.0 0
0.5488116360940264


0.5488116360940264

In [106]:
from nltk.translate.bleu_score import sentence_bleu

In [108]:
candidate = 'I I I I I.'.split()
print('BLEU score -> {}'.format(sentence_bleu(references, candidate )))

BLEU score -> 3.0199442872921993e-234


/heavy_data/jkfirst/workspace/git/publish/env_pub/lib/python3.6/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/heavy_data/jkfirst/workspace/git/publish/env_pub/lib/python3.6/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/heavy_data/jkfirst/workspace/git/publish/env_pub/lib/python3.6/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram o

In [110]:
candidate = 'you love I always.'.split()
print('BLEU score -> {}'.format(sentence_bleu(references, candidate )))

BLEU score -> 5.548953570484417e-235


In [111]:
candidate = 'know that I always love you that much.'.split()
print('BLEU score -> {}'.format(sentence_bleu(references, candidate )))

BLEU score -> 3.706738513091155e-233
